In [1]:
# Import necessary libraries
import pandas as pd
from datetime import datetime as dt

In [2]:
# Load data and print its shape

# Train data
train = pd.read_csv("data/train.csv")

# Test data 
test = pd.read_csv("data/test.csv")

# Print shape
print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (100000, 16)
test: (25000, 16)


In [3]:
# Drop unnecessary columns
def drop_unnecessary_columns(data, columns):
    return data.drop(columns, axis=1)

columns_to_drop = ["message_id", "reply_to_id", "text", "forwarded_from", "caption", "media_size",
                   "photo_resolution", "media_duration", "sticker_emoji"]

train = drop_unnecessary_columns(train, columns_to_drop)
test = drop_unnecessary_columns(test, columns_to_drop)

In [4]:
# Extract {day_of_week, day_of_month, hour_of_day, month_of_year} from message_date and drop message_date
def extract_and_delete(data):
    location = data.columns.get_loc('message_date')
    day_of_week = data["message_date"].apply(lambda x: dt.utcfromtimestamp(x).strftime("%w"))
    data.insert(loc=location, column="day_of_week", value=day_of_week)

    day_of_month = data["message_date"].apply(lambda x: dt.utcfromtimestamp(x).strftime("%d"))
    data.insert(loc=location+1, column="day_of_month", value=day_of_month)

    hour_of_day = data["message_date"].apply(lambda x: dt.utcfromtimestamp(x).strftime("%H"))
    data.insert(loc=location+2, column="hour_of_day", value=hour_of_day)

    month_of_year = data["message_date"].apply(lambda x: dt.utcfromtimestamp(x).strftime("%m"))
    data.insert(loc=location+3, column="month_of_year", value=month_of_year)
    
    return data.drop(['message_date'], axis=1)


train = extract_and_delete(train)
test = extract_and_delete(test)

In [5]:
# Replace text_emojis column with contains_emoji
def contains_emoji(data):
    location = data.columns.get_loc("text_length")
    data.insert(loc=location+1, column="contains_emoji", value=data["text_emojis"].apply(lambda x: 1 if type(x) == str else 0))
    
    return data.drop("text_emojis", axis=1)

train = contains_emoji(train)
test = contains_emoji(test)

In [6]:
# Replace reply_to_sender column with reply_to as reply_to_id doesn't exist no more
def rename_reply_column(data):
    return data.rename(columns={"reply_to_sender": "reply_to"})

train = rename_reply_column(train)
test = rename_reply_column(test)